# Simple Integration

This lesson is a showcase for setting up a simple integration with `simframe`. And explains how to write/read data files.

The goal is to integrate the following differential equation

$\frac{\mathrm{d}Y}{\mathrm{d}x} = b Y$

with the initial condition

$Y \left( 0 \right) = A$.

The solution of this problem is given by

$Y \left( x \right) = A \cdot \exp \left( bx \right)$.

## Setting the problem parameters

In [ ]:
A  = 10. # Initial value
b  = -1. # decay scale
dx = 0.1 # Stepsize

## Initializing the simulation frame

In [ ]:
from simframe import Frame

In [ ]:
sim = Frame(description="Simple Integration")

In [ ]:
sim

This created an empty framework `sim`.

As you can see the Integrator and Writer of the Frame object are not set initially. The Writer contains the instructions for writing output files. If you do not want to outputs you can ommit setting it.

The Integrator, on the other hand, has to be set for a successful execution. At the minimum it has to control when to stop the simulation.

## Setting the Writer

`simframe` comes with a built-in writer for writing outputs in the `.hdf5` file format using the `h5py` module.

In [ ]:
from simframe import writers

In [ ]:
writers.hdf5writer

We want to change the name of the output directory and we want to set the writer to overwrite existing files. By default the writer will raise an exception if the file it needs to write already exists to protect your data.

In [ ]:
writers.hdf5writer.datadir = "1_data"
writers.hdf5writer.overwrite = True

In [ ]:
writers.hdf5writer

The writer can be simply assigned to the frame object

In [ ]:
sim.writer = writers.hdf5writer

In [ ]:
sim

## Adding fields

We now have to add fields to our simulation frame, which has a method for this task.

Here we add the field for our variable `Y` and initialize it with the initial condition `A`.\
The value you set has to be of the correct shape and data type. It cannot be changed later.

In [ ]:
sim.addfield("Y", A)

In [ ]:
sim

The field `Y` can be easily accessed and manipulated. All `NumPy` functions for `ndarray` work on it.

For example:

In [ ]:
sim.Y

In [ ]:
sim.Y * 2

In [ ]:
import numpy as np

np.exp(sim.Y)

We now have to add a field for `x`. But since this is our integration variable, we have to add a special field for it.\
Here we set the initial value to `0`:

In [ ]:
sim.addintegrationvariable("x", 0.)

In [ ]:
sim

An integration variable is a special field that has additional functionality for the stepsize of our simulation.

## Adding functions

First we define a function that controls the stepsize of the simulation. It needs to have the frame object as argument. Therefore, anything within the object can be accessed. The function needs to return the value of the stepsize

In this simple integration, we want to have a constant stepsize, that we defined above.

In [ ]:
def fdx(sim):
    return dx

We can now simply assign this function to the integration variable

In [ ]:
sim.x.updater = fdx

Furthermore, the integration variable needs to know the desired snapshots. Snapshots define the value of the integration variable at which an output should be written. Even if you do not want to write output files the list of snapshots needs to contain at least one value, which defines when the simulation should be stopped.

Here we want to have 20 output files written between `x=0.5` and `x=10.` and assign them to the integration variable.

In [ ]:
snaps = np.linspace(0.5, 10., 20)
sim.x.snapshots = snaps

The integration variable has now anything it needs to access its additional functionality which are the stepsize,

In [ ]:
sim.x.updater.updater.update(sim)

In [ ]:
sim.x.stepsize

the maximum possible stepsize, i.e., the stepsize until the next snapshot,

In [ ]:
sim.x.maxstepsize

and the value of the next snapshot, which is here identical to the maximum stepsize.

In [ ]:
sim.x.nextsnapshot

## Setting the differential equation

We also have to define the differential equation of our variable `Y`, that we want to integrate. The function needs as argument the frame object and the variable itself.

In [ ]:
def dYdx(sim, Y):
    return b*Y

We can simply assign it to the variable as before.

In [ ]:
sim.Y.differentiator = dYdx

## Setting the integrator

Last but not least we have to specify the integrator which controls the simulation. The integrator needs the integration variable as argument upon initialization.

In [ ]:
from simframe import Integrator

sim.integrator = Integrator(sim.x, description="Simple first order integration")

In [ ]:
sim

If we ran the simulation now, the output files would be written, but `Y` would stay constant. The reason is that the integrator needs a set of instructions that specifiy what to do.

An instruction specifies the integration scheme, the variable to be integrated, and the fraction of the stepsize it should operate on. `simframe` provides a simple Euler 1st order scheme. We'll explain later how to create your own schemes.

In [ ]:
from simframe import Instruction
from simframe import schemes

Here we set up our instruction for integrating `Y` with a simple Euler 1st order scheme by using the full timestep.

In [ ]:
instructions = [Instruction(schemes.expl_1_euler, sim.Y)]

We simply add this list to our integrator.

In [ ]:
sim.integrator.instructions = instructions

## Running the simulation

The simulation is now ready to go!

In [ ]:
sim.run()

Since the data directory did not exist before, it was created by `simframe` and the output files have been written.

## Reading the outputs

Every writer should provide a reader for its specific file format. It can either list all data files in the data directory,

In [ ]:
sim.writer.read.listfiles()

read a single output file,

In [ ]:
output3 = sim.writer.read.output(3)

or read the full data set.

In [ ]:
data = sim.writer.read.all()

The data is easily accesible from the framework namespace, e.g.

In [ ]:
data.x

## Fitting of the data

This is a simple fit of the data to see if the original values of `A` and `b` can be obtained.

In [ ]:
def fitfunc(x, A, b):
    return A*np.exp(b*x)

In [ ]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(fitfunc, data.x, data.Y)

In [ ]:
from IPython.display import Markdown as md
md("| |Simulation|Exact|\n|-|-|-|\n|A|{:4.2f}|{:4.2f}|\n|b|{:4.2f}|{:4.2f}|".format(popt[0],A,popt[1],b))

You can re-run the simulation and play around with `A`, `b`, and `dx` to see how this simple integration behaves. Note: `dx` will never be larger than the distance to the next snapshot. If you want to increase `dx` beyond that you have to reduce the number of snapshots.